In [1]:
from utils import *
import tensorflow as tf
from keras import backend as K
from keras.optimizers import SGD
from keras.layers import CuDNNLSTM, Dense

2022-10-23 15:37:55.836869: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-23 15:37:55.947872: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-23 15:37:56.417825: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/moises/workspace/devel/lib
2022-10-23 15:37:56.417878: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.

Check if keras is using GPU

In [2]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
K._get_available_gpus()

2022-10-23 15:37:57.144247: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-23 15:37:57.165367: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-23 15:37:57.181656: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-23 15:37:57.181925: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5



2022-10-23 15:37:57.563771: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-23 15:37:57.563982: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-23 15:37:57.564106: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-23 15:37:57.564203: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 387 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5
2022-10-23 15:37:57.565347: I tensorflow/stream_executor/cuda/cuda_gpu_executor

['/device:GPU:0']

2022-10-23 15:37:57.565470: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-23 15:37:57.565596: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-23 15:37:57.565903: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-23 15:37:57.566008: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-23 15:37:57.566116: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

Get tickers

In [3]:
tickers_sets_path = 'data/tickers/sets/'

sets_files = sorted([tickers_sets_path + x for x in listdir(tickers_sets_path) if 'json' in x], reverse=False)

tickers_sets = []

for set_file in sets_files:
    with open(set_file, 'r') as f:
        tickers_sets.append(json.load(f))

Read dataset

In [4]:
df = pd.read_csv('data/dataset.csv')
df = df.set_index('Date')
# df = df[:-60]

Define training sets and data size

In [5]:
in_sets = [0,1,2,3,4]
out_sets = [4]
data_size = 340

input_tickers = set()
output_tickers = set()
for in_set in in_sets:
    input_tickers = input_tickers.union(set(tickers_sets[in_set]))

for out_set in out_sets:
    output_tickers = output_tickers.union(set(tickers_sets[out_set]))

input_tickers = list(input_tickers)


Process data

In [6]:
scaler, x, y, dic = dataa(df, data_size, input_tickers, output_tickers, \
                        step_size=1, input_size=20, output_size=20, \
                        feature_range=(0,1), return_log=False)

501 1


Split dataset i training, validation and test

In [7]:
x_train, x_val, y_train, y_val = train_test_split(x,y,train_size=0.20)

Define hyperparameters

In [8]:
input_shape = (x.shape[1],x.shape[2])
output_shape = (y.shape[1], y.shape[2])
layers_info_list = [
    [
        Dense(units=10, \
            input_shape=input_shape, activation='relu'),
        Dense(units=1, activation='relu')
    ],
    [
        Dense(units=10, \
            input_shape=input_shape, activation='sigmoid'),
        Dense(units=1, activation='sigmoid')
    ],
    [
        Dense(units=25, \
            input_shape=input_shape, activation='relu'),
        Dense(units=1, activation='relu')
    ],
    [
        Dense(units=25, \
            input_shape=input_shape, activation='sigmoid'),
        Dense(units=1, activation='sigmoid')
    ]
]


Create Models

In [9]:
models = []
hyperparameters = range(len(layers_info_list))

for i in hyperparameters:
    layers_info = layers_info_list[i]
    models.append(create_model2(output_shape, layers_info, optimizer=SGD(learning_rate=0.01) ,metrics=['accuracy', 'mse']))

Train networks

In [10]:
history_list = []
epochs = 300

set_name = 'set4/'
plot_path_dir = 'metrics/' + set_name
extra_info = ''

j=0
for i in hyperparameters:
    try:
        model = models[j]
        j += 1
        
        history = model.fit(
            x=x_train,
            y=y_train,
            epochs=epochs,
            batch_size=10,
            validation_data=(x_val, y_val)
        )

        history_list.append(history)

        model_name = 'LSTM' + str(i) + extra_info
        # model.save('models/' + set_name + model_name + '/model')

        plot_path_loss = plot_path_dir + 'LOSS_' + model_name + '.png'
        validation_plot(history.history['loss'], history.history['val_loss'], 'train vs validation loss', f=lambda: plt.savefig(plot_path_loss))

        plot_path_mse = plot_path_dir + 'MSE_' + model_name + '.png'
        validation_plot(history.history['mse'], history.history['val_mse'], 'train vs validation mse', f=lambda: plt.savefig(plot_path_mse), plot_type='mse')

        plot_path_accuracy = plot_path_dir + 'ACCURACY_' + model_name + '.png'
        validation_plot(history.history['accuracy'], history.history['val_accuracy'], 'train vs validation accuracy', f=lambda: plt.savefig(plot_path_accuracy), plot_type='accuracy')
    except:

        print('ERRO NO MODELO ' + str(i))
    

Epoch 1/300


2022-10-23 15:37:59.157085: E tensorflow/stream_executor/cuda/cuda_blas.cc:218] failed to create cublas handle: cublas error
2022-10-23 15:37:59.157111: E tensorflow/stream_executor/cuda/cuda_blas.cc:220] Failure to initialize cublas may be due to OOM (cublas needs some free memory when you initialize it, and your deep-learning framework may have preallocated more than its fair share), or may be because this binary was not built with support for the GPU in your machine.
2022-10-23 15:37:59.157125: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at matmul_op_impl.h:620 : INTERNAL: Attempting to perform BLAS operation using StreamExecutor without BLAS support


ERRO NO MODELO 0
Epoch 1/300
ERRO NO MODELO 1
Epoch 1/300


2022-10-23 15:37:59.592754: E tensorflow/stream_executor/cuda/cuda_blas.cc:218] failed to create cublas handle: cublas error
2022-10-23 15:37:59.592776: E tensorflow/stream_executor/cuda/cuda_blas.cc:220] Failure to initialize cublas may be due to OOM (cublas needs some free memory when you initialize it, and your deep-learning framework may have preallocated more than its fair share), or may be because this binary was not built with support for the GPU in your machine.
2022-10-23 15:37:59.592787: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at matmul_op_impl.h:620 : INTERNAL: Attempting to perform BLAS operation using StreamExecutor without BLAS support


ERRO NO MODELO 2
Epoch 1/300


2022-10-23 15:37:59.852955: E tensorflow/stream_executor/cuda/cuda_blas.cc:218] failed to create cublas handle: cublas error
2022-10-23 15:37:59.852980: E tensorflow/stream_executor/cuda/cuda_blas.cc:220] Failure to initialize cublas may be due to OOM (cublas needs some free memory when you initialize it, and your deep-learning framework may have preallocated more than its fair share), or may be because this binary was not built with support for the GPU in your machine.
2022-10-23 15:37:59.852992: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at matmul_op_impl.h:620 : INTERNAL: Attempting to perform BLAS operation using StreamExecutor without BLAS support


ERRO NO MODELO 3


2022-10-23 15:38:00.114320: E tensorflow/stream_executor/cuda/cuda_blas.cc:218] failed to create cublas handle: cublas error
2022-10-23 15:38:00.114340: E tensorflow/stream_executor/cuda/cuda_blas.cc:220] Failure to initialize cublas may be due to OOM (cublas needs some free memory when you initialize it, and your deep-learning framework may have preallocated more than its fair share), or may be because this binary was not built with support for the GPU in your machine.
2022-10-23 15:38:00.114350: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at matmul_op_impl.h:620 : INTERNAL: Attempting to perform BLAS operation using StreamExecutor without BLAS support
